# HEP4LGT Unit 5: Computing algorithms and HPC

## Markov Chain Monte Carlo example

This notebook contains examples for Unit 5 lecture 1 "Markov Chain Monte Carlo".

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline


## Transition probability matrix

In [ ]:
# columns: "from" state
# raws: "to" state
T = np.array( [
    [ 7/10, 1/3, 1/6 ],
    [ 3/20, 1/3, 1/3 ],
    [ 3/20, 1/3, 1/2 ]
] )


## Dynamical process

Three rooms: states 0, 1, 2. There is an ant in one room and it can transition to another room with some probability. Consider the transitions happening in discrete steps. We run the process for some number of steps and record how many times each state was visited.


In [ ]:
# set random seed for reproducibility
np.random.seed( 1 )


In [ ]:
# run transitions between three states with given transition probability matrix
# Input:
# T -- 3x3 Markov matrix (columns sum to 1)
# Nsteps -- how many steps to make
# start_state =0, 1 or 2
# Output:
# a list with visited states
def run_transitions( T, Nsteps, start_state=0 ):

    # start in state 0
    state = start_state

    visited = []

    for i in range( Nsteps ):

        # read the transition probabilities
        tij = T.transpose()[state]

        # attempt a transition
        r = np.random.random()
        if r < tij[0]:
            state = 0
        elif r < tij[0] + tij[1]:
            state = 1
        else:
            state = 2

        visited.append( state )

    return visited


# count how many times states were visited
# Input:
# list with visited states, containing only 0, 1 or 2
# Output:
# array of three elements with how many times each state was visited
def count_states( visited ):
    
    frequency = np.zeros( 3 )
    for i in range( len( visited  ) ):
        frequency[ visited[i] ] += 1
        
    return frequency




Run the process, count how many times each state was visited, normalize to define _probability_ of the state.

In [ ]:
# try Nsteps = 10, 100, 1000, 10000, 100000
Nsteps = 10
vis = run_transitions( T, Nsteps )
frequency = count_states( vis )

print( "Frequency:")
print( frequency )
print()
print( "Probability:")
print( frequency / frequency.sum() )


In [ ]:
plt.bar( [0,1,2], frequency, width=1, color='skyblue', edgecolor='black')

## Dynamical process vs ensemble

What happens if we act with the transition matrix on a probability vector?

In [ ]:
# starting ensemble
p0 = np.array( [0,0,1] )

print( "Starting ensemble:" )
p = p0.copy()
print(p)
print()

print( "Ensemble evolution:" )
nrpt = 1 # try 1, 2, 3, 5, 10, 20, 30
for i in range( nrpt ):
    p = T.dot(p)
    print(p)


# Eigenvalues and eigenvectors of the transition matrix 

The equilibrium distribution is the eigenvector of the Markov matrix corresponding to the largest eigenvalue, equal 1, of the Markov matrix.

In [ ]:
eigvals, eigvecs = np.linalg.eig( T )

print( eigvals )
print( eigvecs )
print()

idx = eigvals.argsort()[::-1]   
eigvals = eigvals[idx]
eigvecs = eigvecs[:,idx]

print( "Largest eigenvalue:", eigvals[0] )

evec0 = eigvecs.transpose()[0]
print( "Its eigenvector:", evec0 )


evec0 = np.abs( evec0 )
evec0 /= evec0.sum()
print( "Eigenvector properly normalized:", evec0 )


## From weights to transition matrix

Now, all we know are the relative weights of the states and we want to generate a sequence (Markov chain) where the states appear with frequency (i.e. probability) proportional to the weights.

In [ ]:
weights = evec0 / evec0[0]
print( weights )

Try several proposal probability matrices.

In [ ]:
# proposal probability
# columns: "from" state
# rows: "to" state
T0_list = [ \
    np.array( [ \
    [ 1/3, 1/3, 1/3 ], \
    [ 1/3, 1/3, 1/3 ], \
    [ 1/3, 1/3, 1/3 ]  \
    ] ), \
    np.array( [ \
    [ 1/2, 1/2, 1/2 ], \
    [ 1/4, 1/4, 1/4 ], \
    [ 1/4, 1/4, 1/4 ]  \
    ] ), \
    np.array( [ \
    [ 1/2, 2/3, 1/8 ], \
    [ 1/4, 1/6, 3/8 ], \
    [ 1/4, 1/6, 1/2 ]  \
    ] ), \
    np.array( [ \
    [ 1/10, 1/10, 1/10 ], \
    [ 8/10, 8/10, 8/10 ], \
    [ 1/10, 1/10, 1/10 ] \
    ] ), \
    np.array( [ \
    [ 80/179, 80/179, 80/179 ], \
    [ 45/179, 45/179, 45/179 ], \
    [ 54/179, 54/179, 54/179 ]  \
    ] ) \
]

T0 = T0_list[0]


Construct acceptance probability and full transition matrix.

In [ ]:
# acceptance probability
Ta = np.zeros( (3,3) )

for j in range(3):
    for i in range(3):
        prob = weights[i]/weights[j]
        prob *= T0[j,i] / T0[i,j]
        Ta[i,j] = ( prob if prob < 1 else 1 )

# transition probability
newT = np.zeros( (3,3) )

for j in range(3):
    for i in range(3):
        if i==j:
            newT[i,j] = 0
        else:
            newT[i,j] = Ta[i,j]*T0[i,j]
    newT[j,j] = 1 - np.sum( newT[:,j] )


In [ ]:
print( newT )

Run Markov Chain Monte Carlo with new transition matrix.

In [ ]:
# try Nsteps = 10, 100, 1000, 10000, 100000
vis = run_transitions( newT, 100000 )
frequency = count_states( vis )

print( "Frequency:")
print( frequency )
print()
print( "Probability:")
print( frequency / frequency.sum() )


Eigenvalues and eigenvectors.

In [ ]:
eigvals, eigvecs = np.linalg.eig( newT )

print( eigvals )
print( eigvecs )
print()

idx = eigvals.argsort()[::-1]   
eigvals = eigvals[idx]
eigvecs = eigvecs[:,idx]

print( "Largest eigenvalue:", eigvals[0] )

evec0 = eigvecs.transpose()[0]
print( "Its eigenvector:", evec0 )


evec0 = np.abs( evec0 )
evec0 /= evec0.sum()
print( "Eigenvector properly normalized:", evec0 )


What about the acceptance probabilities?

In [ ]:
print( Ta )